In [ ]:
!git clone https://github.com/AjumaML/Hand_Recognition_Project.git

Cloning into 'Hand_Recognition_Project'...
remote: Enumerating objects: 2424, done.
remote: Counting objects: 100% (2424/2424), done.
remote: Compressing objects: 100% (2416/2416), done.
remote: Total 2424 (delta 7), reused 2420 (delta 6), pack-reused 0
Receiving objects: 100% (2424/2424), 3.75 MiB | 34.89 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
#Importing libraries

import numpy as np
from imutils import paths
import os

In [ ]:
imagePaths = list(paths.list_images('/content/Hand_Recognition_Project/dataset'))

In [ ]:
imagePaths[1000]

'/content/Hand_Recognition_Project/dataset/Volume_Up/486.jpg'

In [ ]:
label = imagePaths[0].split(os.path.sep)[-2]
label

'Volume_Down'

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
data = []
labels = []

IMG_SIZE = 224
CHANNELS = 3

for imagePath in imagePaths:
  label = imagePath.split(os.path.sep)[-2]
  image = load_img(imagePath, target_size=(IMG_SIZE, IMG_SIZE))
  image = img_to_array(image)
  image = image/255

  data.append(image)
  labels.append(label)


data = np.array(data, dtype="float32")
labels = np.array(labels)

In [ ]:
data.shape

(2400, 224, 224, 3)

In [ ]:
labels.shape

(2400,)

In [ ]:
labels


array(['Volume_Down', 'Volume_Down', 'Volume_Down', ..., 'play', 'play',
       'play'], dtype='<U11')

In [ ]:
np. unique(labels, return_counts=True)

(array(['Volume_Down', 'Volume_Up', 'play', 'stop'], dtype='<U11'),
 array([900, 900, 300, 300]))

In [ ]:
# Preprocessing the label by encoding them

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labels = le.fit_transform(labels)

In [ ]:
labels[100]

0

In [ ]:
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels)

In [ ]:
labels

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [ ]:
# Splitting the dataset into train and test

from sklearn.model_selection import train_test_split
(trainX, testX, trainY, testY) = train_test_split(data, labels,	test_size=0.20, stratify=labels, random_state=42)

In [ ]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Input

In [ ]:
feature_extractor_layer = ResNet50V2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(IMG_SIZE,IMG_SIZE,CHANNELS)))

94668760/94668760 [==============================] - 6s 0us/step


In [ ]:
feature_extractor_layer.trainable = False

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

In [ ]:
# Model building

model = tf.keras.Sequential()
model.add(feature_extractor_layer)
model.add(layers.Flatten(name="flatten"))
model.add(layers.Dense(1024, activation='relu', name='hidden_layer'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax', name='output'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 hidden_layer (Dense)        (None, 1024)              102761472 
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 output (Dense)              (None, 4)                 4100      
                                                                 
Total params: 126,330,372
Trainable params: 102,765,572
Non-trainable params: 23,564,800
_________________________________________________________________


In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
  loss="categorical_crossentropy",
  metrics=["accuracy"])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
history = model.fit(aug.flow(trainX, trainY),
	                  validation_data=(testX, testY),
	                  epochs=10)

Epoch 1/10
60/60 [==============================] - 33s 377ms/step - loss: 0.2506 - accuracy: 0.9281 - val_loss: 1.9482e-05 - val_accuracy: 1.0000
Epoch 2/10
60/60 [==============================] - 24s 399ms/step - loss: 0.0130 - accuracy: 0.9937 - val_loss: 9.1398e-06 - val_accuracy: 1.0000
Epoch 3/10
60/60 [==============================] - 23s 381ms/step - loss: 0.0125 - accuracy: 0.9953 - val_loss: 3.2803e-06 - val_accuracy: 1.0000
Epoch 4/10
60/60 [==============================] - 23s 389ms/step - loss: 0.0022 - accuracy: 0.9995 - val_loss: 1.3388e-06 - val_accuracy: 1.0000
Epoch 5/10
60/60 [==============================] - 24s 395ms/step - loss: 0.0039 - accuracy: 0.9979 - val_loss: 9.1436e-06 - val_accuracy: 1.0000
Epoch 6/10
60/60 [==============================] - 22s 363ms/step - loss: 0.0027 - accuracy: 0.9990 - val_loss: 2.3918e-06 - val_accuracy: 1.0000
Epoch 7/10
60/60 [==============================] - 23s 385ms/step - loss: 0.0021 - accuracy: 0.9995 - val_loss: 2.128

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
predIdxs = model.predict(testX)

15/15 [==============================] - 2s 91ms/step


In [ ]:
predIdxs[0]

array([2.8353731e-11, 1.0000000e+00, 3.4977379e-09, 3.1036922e-09],
      dtype=float32)

In [ ]:
predIdxs = np.argmax(predIdxs, axis=1)

In [ ]:
print(classification_report(testY.argmax(axis=1), predIdxs,target_names=le.classes_))

              precision    recall  f1-score   support

 Volume_Down       1.00      1.00      1.00       180
   Volume_Up       1.00      1.00      1.00       180
        play       1.00      1.00      1.00        60
        stop       1.00      1.00      1.00        60

    accuracy                           1.00       480
   macro avg       1.00      1.00      1.00       480
weighted avg       1.00      1.00      1.00       480



In [ ]:
model.save('model_gesture_recognition.h5', save_format="h5")

In [ ]:
model = tf.keras.models.load_model('/content/model_hand_gesture.h5')

In [ ]:
image = load_img('/content/20221230_214250.jpg',target_size=(IMG_SIZE, IMG_SIZE))
image = img_to_array(image)
image = image/255

In [ ]:
image.shape

(224, 224, 3)

In [ ]:
pred = model.predict(image.reshape(1,224,224,3))

1/1 [==============================] - 1s 1s/step


In [ ]:
pred

array([[0.99231637, 0.00768364]], dtype=float32)

In [ ]:
np.argmax(pred, axis=1)

array([0])

In [ ]:
le.classes_

array(['play', 'stop'], dtype='<U4')